In [1]:
from  openai import OpenAI # type: ignore
import minsearch # type: ignore
import json
import textwrap
from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError
from tqdm.auto import tqdm
import tiktoken 
import os

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# apikey = open('txt.txt').read()
apikey = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=apikey)

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
# es_client.indices.create(index=index_name, body=index_settings)
try:
    es_client.indices.get(index=index_name)
    print(f"{index_name} already exists")
    # es.indices.delete(index=INDEX_NAME, ignore=[400, 404]) <- uncomment if you want to remove the index.
except NotFoundError:
    response = es_client.indices.create(index=index_name, body=index_settings)
    print(response)

course-questions already exists


In [6]:
def elastic_search_hw(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    result_score = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_score.append(hit['_score'])

    return result_docs, result_score
query_hw = "How do I execute a command in a running docker container?"
hw, hw_score = elastic_search_hw(query_hw)

In [7]:
hw_score

[84.050095, 51.04628, 49.938507]

In [8]:
hw

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [9]:
context_template_hw = """
Q: {question}
A: {text}
""".strip()

prompt_template_hw = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [11]:
context_hw = ''
for doc in hw:
    context_hw = (context_hw + 
    f"Q:{doc['question']}\n\nA: {doc['text']}\n\n")

prompt_wh = prompt_template_hw.format(question=query_hw, context=context_hw).strip()
len(prompt_wh)

1462

In [14]:
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens_num_hw = encoding.encode(prompt_wh)
len(tokens_num_hw)

322